In [6]:
# ====================================================
# import
# ====================================================
import operator
from typing import Annotated, Any, Optional

from dotenv import load_dotenv
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


In [7]:
# ====================================================
# データ構造の定義
# ====================================================

# ペルソナを表すデータモデル
class Persona(BaseModel):
    name: str = Field(description="ペルソナの名前")
    background: str = Field(description="ペルソナの持つ背景")


# ペルソナのリストを表すデータモデル
class Personas(BaseModel):
    personas: list[Persona] = Field(default_factory=list, description="ペルソナのリスト")


# インタビュー内容を表すデータモデル
class Interview(BaseModel):
    persona: Persona = Field(description="インタビュー対象のペルソナ")
    question: str = Field(description="インタビューでの質問")
    answer: str = Field(description="インタビューでの回答")


# インタビュー結果のリストを表すデータモデル
class InterviewResult(BaseModel):
    interviews: list[Interview] = Field(default_factory=list, description="インタビュー結果のリスト")


# 評価の結果を表すデータモデル
class EvaluationResult(BaseModel):
    reason: str = Field(description="判断の理由")
    is_sufficient: bool = Field(description="情報が十分かどうか")


In [8]:
# 要件定義書AIエージェントのステート
class InterviewState(BaseModel):
    user_request: str = Field(description="ユーザーからのリクエスト")
    personas: Annotated[list[Persona], operator.add] = Field(
        default_factory=list, description="生成されたペルソナのリスト"
    )
    interviews: Annotated[list[Interview], operator.add] = Field(
        default_factory=list, description="実施されたインタビューのリスト"
    )
    requirements_doc: str = Field(default="", description="生成された要件定義")
    iteration: int = Field(default=0, description="ペルソナ生成とインタビューの反復回数")
    is_information_sufficient: bool = Field(default=False, description="情報が十分かどうか")


## 主要コンポーネントの実装


In [9]:
# ====================================================
# PersonaGenerator：ペルソナを生成する
# ====================================================
class PersonaGenerator(BaseModel):
    def __init__(self, llm: ChatOpenAI, k: int = 5):
        self.llm = llm.with_structured_output(Personas)
        self.k = k

    def run(self, user_request: str) -> Personas:
        # プロンプトテンプレートを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザーインタビュー用の多様なペルソナを作成する専門家です。",
                ),
                (
                    "human",
                    f"以下のユーザーリクエストに関するインタビュー用に、{self.k}人のペルソナを生成してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n\n"
                    "各ペルソナには名前と簡単な背景を含めてください。年齢、性別、職業、技術的専門知識において多様性を確保してください。",
                ),
            ]
        )
        # ペルソナ作成のためのチェーンを作成
        chain = prompt | self.llm
        # ペルソナを作成
        return chain.invoke({"user_request": user_request})


In [10]:
# ====================================================
# InterviewConductor: ペルソナにインタビューを実施する
# ====================================================
class InterviewConductor(BaseModel):
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm

    def run(self, user_request: str, personas: list[Persona]) -> InterviewResult:
        # 質問を生成
        questions = self._generate_questions(user_request=user_request, personas=personas)
        # 回答を生成
        answers = self._generate_answers(personas=personas, questions=questions)
        # 質問と回答の組合せからインタビューリストを作成
        interviews = self._create_interviews(personas=personas, questions=questions, answers=answers)
        # インタビュー結果を返す
        return InterviewResult(interviews=interviews)

    def _generate_questions(self, user_request: str, personas: list[Persona]) -> list[str]:
        # 質問生成のためのプロンプトを定義
        question_prompt = ChatPromptTemplate.from_messages(
            [
                (
                    "system",
                    "あなたはユーザー要件に基づいて、適切な質問を生成する専門家です。",
                ),
                (
                    "human",
                    "以下のペルソナに関連するユーザーリクエストについて、1つの設問を生成してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n"
                    "ペルソナ：{persona_name} - {persona_background}\n\n"
                    "質問は具体的で、このペルソナの視点から重要な情報を引き出すように設計してください。",
                ),
            ]
        )
        # 質問生成のためのチェーンを作成
        question_chain = question_prompt | self.llm | StrOutputParser()

        # 各ペルソナに対する質問クエリを生成
        question_queries = [
            {
                "user_request": user_request,
                "persona_name": persona.name,
                "persona_background": persona.background,
            }
            for persona in personas
        ]
        # 質問をバッチ処理で生成
        return question_chain.batch(question_queries)

    def _generate_answers(self, personas: list[Persona], questions: list[str]) -> list[str]:
        # 回答生成のためのプロンプトを定義
        answer_prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは以下のペルソナとして回答しています：{persona_name} - {persona_background}"),
                (
                    "human",
                    "以下の質問に対して、ペルソナの視点から回答してください。\n\n" "質問：{question}\n" "回答：",
                ),
            ]
        )
        # 回答生成のためのチェーンを作成
        answer_chain = answer_prompt | self.llm | StrOutputParser()

        # 各ペルソナに対する回答クエリを作成
        answer_queries = [
            {
                "persona_name": persona.name,
                "persona_background": persona.background,
                "question": question,
            }
            for persona, question in zip(personas, questions, strict=True)
        ]
        # 回答をバッチ処理で生成
        return answer_chain.batch(answer_queries)

    def _create_interviews(self, personas: list[Persona], questions: list[str], answers: list[str]) -> list[Interview]:
        # ペルソナごとに質問と回答の組合せからインタビューオブジェクトを作成
        return [
            Interview(
                persona=persona,
                question=question,
                answer=answer,
            )
            for persona, question, answer in zip(personas, questions, answers, strict=True)
        ]


In [15]:
# ====================================================
# InformationEvaluator: 収集した情報の十分性を評価する
# ====================================================
class InformationEvaluator(BaseModel):
    def __init__(self, llm: ChatOpenAI):
        self.llm = llm.with_structured_output(EvaluationResult)

    def run(self, user_request: str, interviews: InterviewResult) -> EvaluationResult:
        # プロンプトを定義
        prompt = ChatPromptTemplate.from_messages(
            [
                ("system", "あなたは包括的な要件文書を作成するための情報の十分性を評価する専門家です。"),
                (
                    "human",
                    "以下のユーザーリクエストとインタビュー結果に基づいて、包括的な要件文書を作成するのに十分な情報が集まったかどうかを判断してください。\n\n"
                    "ユーザーリクエスト：{user_request}\n\n"
                    "インタビュー結果：{interview_results}",
                ),
            ]
        )
        # 情報の十分性を評価するチェーンを作成
        chain = prompt | self.llm
        # 評価結果を返す
        return chain.invoke(
            {
                "user_request": user_request,
                "interview_results": "\n".join(
                    f"ペルソナ：{interview.persona.name} - {interview.persona.background}\n"
                    f"質問：{interview.question}\n"
                    f"回答：{interview.answer}\n\n"
                    for interview in interviews.interviews
                ),
            }
        )


In [ ]:
# ====================================================
# RequirementsDocumentGenerator: 要件定義書を生成する
# ====================================================
